# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **religious events or traditions** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **religious events or traditions**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **religious events or traditions**?  For this category you might consider calendar events, demographic data about religion in the region and neighboring regions, participation in religious events, or how religious events relate to political events, social movements, or historical events.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\gthampi\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\gthampi\\AppData\\Local\\Temp\\pip-install-49csi1vc\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\gthampi\\AppData\\Local\\Temp\\pip-install-49csi1vc\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: C:\Users\gthampi\AppData\Local\Temp\pip-install-49csi1vc\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [2]:
%matplotlib notebook
import geopandas
import descartes
import mplleaflet as lft
import matplotlib.pyplot as plt
import pandas as pd

geopandas.datasets.available




### create the figure with religiosity


## read in shape file
counties = geopandas.read_file('gz_2010_us_050_00_20m.shp')


## I'm removing puerto rico and the westmost alaskan county as they severely skew the map
counties = counties[counties['STATE']!='72']
counties=counties[counties.bounds['maxx']!=max(counties.bounds['maxx'])]
##create a FIPS code for merging
counties['FIPS'] = counties['STATE'] + counties['COUNTY']
counties['FIPS'] = counties['FIPS'].astype(int)

## read in religiosity data
religion = pd.read_excel('religion.xlsx')


## merge with counties
df = pd.merge(left=counties, right = religion, how = 'left',left_on = 'FIPS' , right_on = 'FIPS')

## read in climate survey data
climate = pd.read_csv('climate.csv', encoding = 'utf-8')

##only retain county level data
climate = climate[climate['GeoType']=='County']
climate['State']=climate['GeoName'].str.extract(r'(, )(.*)')[1]
#map state to two letter state
stateac=pd.read_csv('state acronym.csv', encoding = 'utf-8')
stateac['State'] = stateac['State'].str.strip()

climate=pd.merge(left=climate, right = stateac, how='left' , left_on = 'State', right_on = 'State', indicator = True)

## do some regex to clean up the county names, this specially needed because this file doesn't have fips codes
climate['GeoName'].replace( r',.*' , r'', regex = True, inplace = True)
climate['GeoName'].replace( r' County' , r'', regex = True, inplace = True)
climate['GeoName'].replace( r' Parish' , r'', regex = True, inplace = True)
climate['GeoName'].replace( r' Borough' , r'', regex = True, inplace = True)
climate['GeoName'].replace( r' Census Area' , r'', regex = True, inplace = True)
climate['GeoName'].replace( r' Municipality' , r'', regex = True, inplace = True)

climate['StateCounty'] = climate['GeoName'].str.strip() + climate['Acronym'].str.strip()

## fips code
fips = pd.read_csv('fips.csv', encoding = 'utf-8')
fips['StateCounty'] = fips['Name'].str.strip() + fips['State'].str.strip()


#merge climate and fips

climate = pd.merge(left = climate, right = fips, how = 'left' , left_on = 'StateCounty' , right_on = 'StateCounty')

#merge climate file to the county file
df = pd.merge(left=df, right = climate, how = 'left',left_on = 'FIPS' , right_on = 'FIPS')



ModuleNotFoundError: No module named 'geopandas'

In [ ]:

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.ticker as mtick

plt.clf()

## create plot grid
fig = plt.figure(figsize =(10,10))
ax2 = fig.add_subplot(2,2,2)
ax1 = fig.add_subplot(2,2,1)
ax4 = fig.add_subplot(2,2,4)
ax3 = fig.add_subplot(2,2,3)



#draw first plot
df.plot(ax=ax1,cmap = 'Blues',  column = 'prienvOppose')
## insert its colorbar, the default colorbar is ugly and the plot function doesn't support modifying it, below work around was necessary

mn = df.prienvOppose.min()
mx = df.prienvOppose.max()
norm = Normalize(vmin=mn, vmax=mx)
n_cmap = cm.ScalarMappable(norm=norm, cmap="Blues")
## someone tell me why the below is necessary! 
n_cmap.set_array([])
cbar1=ax1.get_figure().colorbar(n_cmap, ax=ax1, orientation='horizontal', format = "%d" , shrink =  0.75)
ax1.set_title('County population, in percentage \n who oppose prioritizing the environment over the economy', fontdict = {'fontsize' : 8}, color = 'darkblue')
cbar1.ax.tick_params( width=0, labelsize =7)


## draw plot 4  
df.plot(ax=ax4, cmap = 'Reds' , column = 'EVANRATE')
mn = df.EVANRATE.min()
mx = df.EVANRATE.max()
norm = Normalize(vmin=mn, vmax=mx)
n_cmap = cm.ScalarMappable(norm=norm, cmap="Reds")
## someone tell me why the below is necessary! 
n_cmap.set_array([])
cbar=ax4.get_figure().colorbar(n_cmap, ax=ax4, orientation='horizontal', format = "%d", shrink = 0.75 )
ax4.set_title('County population, per thousand \n who are adherents of evangelical christianity', fontdict = {'fontsize' : 8}, color = 'darkred')
ax4.tick_params(axis='both', which='both', length=0)
cbar.ax.tick_params( width=0, labelsize=7)

## draw the scatter plot
ax2.scatter(x=df['EVANRATE'], y=df['prienvOppose'] , s =2, color = 'darkgrey' )
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
ax2.set_title('Evangelical christianity adherents per thousand vs \n percentage opposed to prioritizing the environment, US counties' , fontdict = {'fontsize' : 8}, color = 'black')
ax2.tick_params(axis='both', which='both', length=0)

##give the figure a title
fig.suptitle('A visual of religious belief vs attitude towards the environment \n across counties in the United States', fontsize=16)


ax1.set_axis_off()
ax3.set_axis_off()
ax4.set_axis_off()
plt.savefig('religion v environment.png', facecolor= 'white')
plt.show()                                